# Weiszfeld Algorithm

Function to determine which store is to be used for the distribution center.

To test the accuracy and/or visualize the stores, use this https://www.mapcustomizer.com/ 
Simply plot the store long and lat in the system.

This app uses Weiszfeld's algorithm. This particular algorithm takes an arbitrary point a. Then, each of the points which are desired for optimization can be defined as a_1, a_2, a_3, ..., a_n. Then, take the a_1/(distance from a to a_1) + a_2/(distance from a to a_2) + ... + a_n/(distance from a to a_n). Divide this value by the sum 1/(distance from a to a_1) + 1/(distance from a to a_2) + ... + a_n/(distance from a to a_n). The resulting point p is the next approximation. From there, repeat the algorithm, except replace a with p. This algorithm repeats until you get a point q that converges, which is then the point that minimizes the sum of the distances to this point from the other points. Using a code of this algorithm with 500 iterations, the app quickly calculates the geometric median of a set of points, meaning the point that minimizes the sum of this point to the other desired location points. The user can input a set of points by pressing on a specific coordinate plane and then see their geometric median.

In [94]:
!pip install gmplot

In [109]:
# Import the data from csv
import pandas as pd
import numpy as np
import math
from numpy import array
from geopy.distance import geodesic
import gmplot
import requests
import json

In [116]:
code = input("Enter country code to analyze:")

# We edited some of the dataset
countries = pd.read_csv (fr"dataset starbucks.csv")
countries = countries[countries.state == code]
df = countries[["latitude","longitude"]]

coordinates=df.values.tolist()

df

Enter country code to analyze:JP


,latitude,longitude
1787,34.995833,135.738770
1788,32.860052,130.784770
1789,34.747309,135.358266
1790,36.232015,137.955864
1791,39.711316,141.099140
...,...,...
13430,34.664610,135.501173
13431,34.707509,135.499631
13433,35.003517,135.769750
13518,36.310211,140.342127


In [117]:
def weiszfeld(points):

    max_error = 0.0000000001

    x=np.array([point[0] for point in  points])
    y=np.array([point[1] for point in  points])


    ext_condition = True

    start_x = np.average(x)
    start_y = np.average(y)

    while ext_condition:

        sod = (((x - start_x)**2) + ((y - start_y)**2))**0.5

        new_x = sum(x/sod) / sum(1/sod)
        new_y = sum(y/sod) / sum(1/sod)

        ext_condition = (abs(new_x - start_x) > max_error) or (abs(new_y - start_y) > max_error)

        start_y = new_y
        start_x = new_x

        center_point = [start_x, start_y]

#     print(start_x, start_y)
    return calc_geodesic_dist(center_point, points)

#Using brute force method, can be optimised.
def calc_geodesic_dist(center_point, points):
    min_dist_from_center = 999999999999999999999999999999
    for point in points:
        if geodesic(point,center_point) < min_dist_from_center:
            min_dist_from_center = geodesic(point,center_point)
            center = point
    return center


if __name__=="__main__":
    # weiszfeld([(2,1), (12,2), (3,9), (13,11)]
#     coordinates = [(49.276774,-123.12523), (49.841375,-119.487573), (49.500151,-117.283269), (49.23206,-123.032935), (51.08277, -114.154081)]
    center = weiszfeld(coordinates)
    print(center)

[35.657433, 139.338649]


# Request the Distance Matrix from Google API

In [49]:
#Call Matrix API using Free source
def call_matrix_api_free(origins, destinations):
    url = 'https://api.distancematrix.ai/maps/api/distancematrix/json'
    key = ''
    params = {'key': key, 'origins': origins, 'destinations': destinations}

    req = requests.get(url=url, params=params)
    response = json.loads(req.content)
    return response

#format the coordinates array to pass to param API
def urlparser(coordinates):
    n = len(coordinates)
    txt1 = [0] * n 
    for i in range(len(coordinates)):
        txt1[i] = ','.join(list(map(str, coordinates[i])))

    txt = '|'.join(txt1)
#     return urllib.parse.quote(txt)
    return txt

In [6]:
n = len(coordinates)
distance_matrix = [[None] * n for i in range(n)]
counter = 0
MAX_LIMIT = 1

#loop through all store
for i in range(n):
    #format the lat and long of the store
    batch_origin = df.iloc[i].values.tolist()
    origins = ','.join(list(map(str, batch_origin)))
    print("Calculating the for the "+str(i)+" coordinate")
    
    #check for every 24 other store since we can only pass 25 location 
    #in 1 request at API. 
    for j in range(math.ceil(n/MAX_LIMIT)):
        #pass <=24 lat long to be the destination
        batch_destination = df.iloc[j*MAX_LIMIT+counter:(j+1)*MAX_LIMIT+counter,:].values.tolist()
        destination = urlparser(batch_destination)
        
        #call the API
        api_response = call_matrix_api_free(origins, destination)
        if api_response['status'] == 'OK':
            y = j * MAX_LIMIT
            for element in api_response['rows'][0]['elements']:
                if element['status'] == 'OK':
                    # A -> B is the same as B -> A for simplicity sake and reduce the request needed
                    distance_matrix[i][y+counter] = element['distance']['value']
                    distance_matrix[y+counter][i] = element['distance']['value']
                    print(element['distance']['value'])
                    y += 1
                else:
                    distance_matrix[i][y] = -1
#                     print(-1)
    counter += 1
        
print(distance_matrix)

Calculating the for the 0 coordinate
0
23965
25639
20638
20293
23792
25997
24976
29820
24675
25169
27025
25244
26777
30096
24105
18651
26551
24136
24799
25303
30495
20238
27232
21047
28194
20495
28168
26104
26245
24416
25977
26389
21086
13165
31246
20485
19948
25130
28742
26005
19146
24542
26822
18395
19225
33564
22444
24454
Calculating the for the 1 coordinate
0
5411
4752
4731
3503
1874
3008
9194
4447
4158
4900
3317
4738
9470
13310
7032
2742
4082
822
13837
9424
10176
4964
4320
9906
6020
5521
1981
3047
932
3010
2266
5882
8630
18868
17131
27157
2163
7671
12014
13212
566
2948
9380
14558
12938
4253
3490
Calculating the for the 2 coordinate
0
1929
2796
4103
4496
3475
4744
2067
8888
4114
2519
3866
5020
8223
5946
2518
2379
4442
21889
14620
6295
4321
1940
5772
3233
4392
3796
3334
4758
3136
4888
2287
8758
24065
12044
22117
3629
12867
6005
9964
4988
3911
12662
9932
8488
4583
2236
Calculating the for the 3 coordinate
0
2210
2908
3302
2281
6154
1409
7694
4536
2549
4288
6430
9359
5388
2473
870
324

0
3933
Calculating the for the 48 coordinate
0
[[0, 23965, 25639, 20638, 20293, 23792, 25997, 24976, 29820, 24675, 25169, 27025, 25244, 26777, 30096, 24105, 18651, 26551, 24136, 24799, 25303, 30495, 20238, 27232, 21047, 28194, 20495, 28168, 26104, 26245, 24416, 25977, 26389, 21086, 13165, 31246, 20485, 19948, 25130, 28742, 26005, 19146, 24542, 26822, 18395, 19225, 33564, 22444, 24454], [23965, 0, 5411, 4752, 4731, 3503, 1874, 3008, 9194, 4447, 4158, 4900, 3317, 4738, 9470, 13310, 7032, 2742, 4082, 822, 13837, 9424, 10176, 4964, 4320, 9906, 6020, 5521, 1981, 3047, 932, 3010, 2266, 5882, 8630, 18868, 17131, 27157, 2163, 7671, 12014, 13212, 566, 2948, 9380, 14558, 12938, 4253, 3490], [25639, 5411, 0, 1929, 2796, 4103, 4496, 3475, 4744, 2067, 8888, 4114, 2519, 3866, 5020, 8223, 5946, 2518, 2379, 4442, 21889, 14620, 6295, 4321, 1940, 5772, 3233, 4392, 3796, 3334, 4758, 3136, 4888, 2287, 8758, 24065, 12044, 22117, 3629, 12867, 6005, 9964, 4988, 3911, 12662, 9932, 8488, 4583, 2236], [20638, 4

In [8]:
# Store the matrix in csv
arr = np.asarray(distance_matrix)
pd.DataFrame(arr).to_csv(fr'StoreDistance/{code} Distance.csv')

# Christofides Approach
Function to find the shortest path for the delivery truck to make an optimal delivery. (Christofides Approach)

See: https://github.com/Retsediv/ChristofidesAlgorithm
https://github.com/geopy/geopy
https://cse442-17f.github.io/Traveling-Salesman-Algorithms/

In [118]:
def tsp(data,dist_mat):
    # build a graph
    G = build_graph(data,dist_mat)
#   print("Graph: ", G)

    # build a minimum spanning tree
    MSTree = minimum_spanning_tree(G)
    # print("MSTree: ", MSTree)

    # find odd vertexes
    odd_vertexes = find_odd_vertexes(MSTree)
    # print("Odd vertexes in MSTree: ", odd_vertexes)

    # add minimum weight matching edges to MST
    minimum_weight_matching(MSTree, G, odd_vertexes)
    # print("Minimum weight matching: ", MSTree)

    # find an eulerian tour
    eulerian_tour = find_eulerian_tour(MSTree, G)

    # print("Eulerian tour: ", eulerian_tour)

    current = eulerian_tour[0]
    path = [current]
    visited = [False] * len(eulerian_tour)
    visited[eulerian_tour[0]] = True
    length = 0

    for v in eulerian_tour:
        if not visited[v]:
            path.append(v)
            visited[v] = True

            length += G[current][v]
            current = v

    length +=G[current][eulerian_tour[0]]
    path.append(eulerian_tour[0])

    print("Result path: ", path)
    print("Result length of the path: {} meter".format(length))

    return length, path

def build_graph(data,dist_mat): # O(n^2)
    graph = {}
    for point1 in range(len(data)):
        for point2 in range(len(data)):
            if point1 != point2:
                if point1 not in graph:
                    graph[point1] = {}

                graph[point1][point2] = dist_mat.iloc[point1,point2]

    return graph


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    import random
    random.shuffle(odd_vert)

    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u

        MST.append((v, closest, length))
        odd_vert.remove(closest)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # print("Neighbours: ", neighbours)

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)

            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):

    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]

    return MatchedMST

if __name__=="__main__":
    dist_mat = pd.read_csv (fr"StoreDistance\{code} Distance.csv")
    dist_mat = dist_mat.iloc[:,1:]
    length, data_sorted = tsp(coordinates,dist_mat)

Result path:  [142, 7, 576, 544, 28, 565, 117, 653, 80, 496, 547, 710, 54, 763, 690, 178, 183, 87, 174, 181, 23, 566, 203, 470, 483, 590, 85, 465, 589, 773, 38, 615, 119, 180, 634, 175, 730, 762, 583, 780, 176, 696, 527, 64, 197, 195, 561, 199, 500, 69, 26, 196, 68, 737, 543, 751, 766, 481, 701, 545, 182, 179, 646, 625, 492, 1, 86, 186, 188, 46, 610, 91, 677, 651, 58, 601, 172, 519, 51, 88, 50, 154, 173, 170, 177, 580, 629, 16, 21, 0, 748, 255, 536, 257, 260, 263, 256, 784, 787, 259, 740, 609, 121, 776, 695, 171, 613, 647, 550, 97, 59, 603, 526, 692, 61, 446, 445, 733, 474, 94, 443, 621, 514, 60, 754, 455, 78, 452, 513, 778, 448, 456, 447, 444, 32, 451, 438, 564, 457, 605, 775, 89, 495, 184, 55, 747, 187, 577, 581, 454, 450, 254, 602, 528, 143, 440, 594, 727, 549, 529, 11, 37, 503, 760, 442, 449, 598, 671, 439, 728, 453, 538, 662, 81, 713, 266, 469, 638, 614, 115, 571, 53, 639, 267, 265, 116, 732, 141, 735, 734, 433, 432, 427, 578, 421, 420, 422, 429, 413, 49, 42, 649, 431, 13, 749, 74

# Plotting the route

In [120]:
# create latitude and longitude with order
n = len(data_sorted)
loc = []

for i in range(n):
    loc.append(coordinates[data_sorted[i]])

store_lats, store_lng = zip(*loc)
center_lats, center_lng = center

In [122]:
gmap = gmplot.GoogleMapPlotter(center_lats, center_lng, 13, apikey="")

# Plot the route taken (straight line):
gmap.plot(store_lats, store_lng, color='red', edge_width=5)

# Mark the distribution center:
gmap.scatter(store_lats, store_lng, color='blue', marker=True)
gmap.marker(center_lats, center_lng, color = 'red', title="Distribution Center")

# Draw the map:
gmap.draw(fr'StoreMap\{code} map.html')